In [1]:
from IPython.core.interactiveshell import InteractiveShell # Nos permite mostar más de una salida por celda
InteractiveShell.ast_node_interactivity = "all" # Nos permite mostar más de una salida por celda

import pandas as pd
import numpy as np
import ast

In [4]:
df_clima = pd.read_csv("../Ficheros/ETL1.csv", index_col=0)
df_attacks = pd.read_csv("../Ficheros/attacks_limpieza_completa.csv", index_col = 0)

En la lección de hoy aprendimos como Crearnos una clase que nos permita limpiar los datos obtenidos de la API.

En este ejercicio, tendréis que crear una clase con el código que usamos en los ejercicios de pair programming de ETL Transformación I.


In [5]:
df_clima.head(1)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,latitud,longitud,paises
0,3,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 2}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 5, 'speed': 2...",26,6,3,1009,none,0,0,295,2,39.78373,-100.445882,USA


In [6]:
df_attacks.head(1)

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,...,fatal_N,fatal_Unknown,fatal_Y,fatal_N.1,fatal_Unknown.1,fatal_Y.1,species_.1,fecha_limpia.1,type.1,age_NORM
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.78373,-100.445882,...,1,0,0,1,0,0,4,7,1,9.116327


In [8]:
class Limpieza:
    
    # Constructor
    
    def __init__(self, dataframe1, dataframe2): 
        
        self.dataframe1 = dataframe1
        self.dataframe2 = dataframe2
        
        
    # Método para poner en minúsculas los datos de las columnas categóricas 
       
    def minuscular(self):
                       
        categoricas = self.dataframe1.select_dtypes(include = object)
             
        for i in categoricas:
                self.dataframe1[i]= self.dataframe1[i].apply(lambda x:x.lower())
        
        return self.dataframe1
    
    
    # Método para filtrar por columnas según una lista de valores  
    def filtrar(self, columna, lista):
        
        self.dataframe1 = pd.DataFrame(self.dataframe1[self.dataframe1[columna].isin(lista)])  
       
        return self.dataframe1
    
    
    # Método para separar columnas con datos tipo json en distintas columnas  
    def separar(self, columna, valor1, valor2):   
        
        self.dataframe2[columna] = self.dataframe2[columna].apply(ast.literal_eval)

        x = self.dataframe2[columna].apply(pd.Series)      
        
        for values in range(len(x.columns)):
            nombre= valor1 + str(x[values].apply(pd.Series)[valor2][0])
            valores = list(x[values].apply(pd.Series)[valor1])
            self.dataframe2.insert(values, nombre, valores)
            
        return self.dataframe2
    
    
    # Método para unir dataframes          
    def unir(self, estrategia, columna1, columna2): 
        
        df = pd.merge(left= self.dataframe1, right = self.dataframe2, how= estrategia, left_on = columna1, right_on = columna2)
        
        return df
        
        
        

In [9]:
instancia = Limpieza(df_attacks, df_clima)


In [10]:
instancia.minuscular()

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,...,fatal_N,fatal_Unknown,fatal_Y,fatal_N.1,fatal_Unknown.1,fatal_Y.1,species_.1,fecha_limpia.1,type.1,age_NORM
0,2018,boating,usa,57.0,white shark,jun,n,f,39.783730,-100.445882,...,1,0,0,1,0,0,4,7,1,9.116327
1,2018,unprovoked,usa,11.0,unespecific,jun,n,f,39.783730,-100.445882,...,1,0,0,1,0,0,3,7,7,3.804860
2,2018,invalid,usa,48.0,unespecific,jun,n,m,39.783730,-100.445882,...,1,0,0,1,0,0,3,7,3,8.403629
3,2018,unprovoked,australia,27.0,unespecific,jun,n,m,-24.776109,134.755000,...,1,0,0,1,0,0,3,7,7,6.313925
4,2018,provoked,mexico,27.0,tiger shark,jun,n,m,23.658512,-102.007710,...,1,0,0,1,0,0,2,7,4,6.313925
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6213,0,unprovoked,australia,27.0,tiger shark,unknown,y,m,-24.776109,134.755000,...,0,0,1,0,0,1,2,14,7,6.313925
6214,0,unprovoked,australia,27.0,tiger shark,unknown,y,m,-24.776109,134.755000,...,0,0,1,0,0,1,2,14,7,6.313925
6215,0,unprovoked,australia,27.0,tiger shark,unknown,y,m,-24.776109,134.755000,...,0,0,1,0,0,1,2,14,7,6.313925
6216,0,unprovoked,panama,27.0,unespecific,unknown,y,m,8.559559,-81.130843,...,0,0,1,0,0,1,3,14,7,6.313925


In [11]:
instancia.filtrar("country", ['usa', 'australia', 'south africa', 'new zealand','papua new guinea'])

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,...,fatal_N,fatal_Unknown,fatal_Y,fatal_N.1,fatal_Unknown.1,fatal_Y.1,species_.1,fecha_limpia.1,type.1,age_NORM
0,2018,boating,usa,57.0,white shark,jun,n,f,39.783730,-100.445882,...,1,0,0,1,0,0,4,7,1,9.116327
1,2018,unprovoked,usa,11.0,unespecific,jun,n,f,39.783730,-100.445882,...,1,0,0,1,0,0,3,7,7,3.804860
2,2018,invalid,usa,48.0,unespecific,jun,n,m,39.783730,-100.445882,...,1,0,0,1,0,0,3,7,3,8.403629
3,2018,unprovoked,australia,27.0,unespecific,jun,n,m,-24.776109,134.755000,...,1,0,0,1,0,0,3,7,7,6.313925
5,2018,unprovoked,australia,27.0,unespecific,jun,n,m,-24.776109,134.755000,...,1,0,0,1,0,0,3,7,7,6.313925
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6210,0,unprovoked,south africa,27.0,unespecific,jul,y,m,-28.816624,24.991639,...,0,0,1,0,0,1,3,6,7,6.313925
6212,0,unprovoked,usa,27.0,unespecific,unknown,n,f,39.783730,-100.445882,...,1,0,0,1,0,0,3,14,7,6.313925
6213,0,unprovoked,australia,27.0,tiger shark,unknown,y,m,-24.776109,134.755000,...,0,0,1,0,0,1,2,14,7,6.313925
6214,0,unprovoked,australia,27.0,tiger shark,unknown,y,m,-24.776109,134.755000,...,0,0,1,0,0,1,2,14,7,6.313925


In [12]:
instancia.separar("rh_profile", "rh", "layer")

,rh950mb,rh900mb,rh850mb,rh800mb,rh750mb,rh700mb,rh650mb,rh600mb,rh550mb,rh500mb,...,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,latitud,longitud,paises
0,2,4,5,6,5,7,9,9,4,1,...,3,1009,none,0,0,295,2,39.783730,-100.445882,USA
1,3,4,6,8,8,9,10,8,2,1,...,4,1010,none,0,0,285,2,39.783730,-100.445882,USA
2,4,5,6,7,9,11,11,8,3,2,...,2,1011,none,0,0,195,2,39.783730,-100.445882,USA
3,3,5,6,9,9,10,10,9,6,6,...,2,1010,none,0,0,155,2,39.783730,-100.445882,USA
4,3,4,6,8,11,10,10,9,8,9,...,3,1009,none,0,0,120,3,39.783730,-100.445882,USA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,14,12,9,1,-2,-3,-4,-4,-4,-4,...,11,1009,rain,3,0,90,3,-5.681607,144.248908,Papua New Guinea
316,13,11,6,0,-2,-3,-4,-4,-4,-4,...,10,1010,rain,3,0,85,4,-5.681607,144.248908,Papua New Guinea
317,12,10,2,-1,-2,-3,-4,-4,-4,-4,...,10,1012,rain,3,0,90,4,-5.681607,144.248908,Papua New Guinea
318,13,10,1,-2,-3,-3,-3,-4,-4,-4,...,10,1011,none,3,0,90,3,-5.681607,144.248908,Papua New Guinea


In [13]:
instancia.separar("wind_profile", "speed", "layer")

,speed950mb,speed900mb,speed850mb,speed800mb,speed750mb,speed700mb,speed650mb,speed600mb,speed550mb,speed500mb,...,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,latitud,longitud,paises
0,2,2,2,3,4,5,5,5,6,6,...,3,1009,none,0,0,295,2,39.783730,-100.445882,USA
1,2,2,2,2,3,4,4,5,5,5,...,4,1010,none,0,0,285,2,39.783730,-100.445882,USA
2,2,3,3,2,3,3,4,4,5,5,...,2,1011,none,0,0,195,2,39.783730,-100.445882,USA
3,2,2,2,3,3,3,3,4,4,5,...,2,1010,none,0,0,155,2,39.783730,-100.445882,USA
4,3,2,2,3,3,3,3,4,4,5,...,3,1009,none,0,0,120,3,39.783730,-100.445882,USA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,4,4,4,3,3,3,3,4,4,4,...,11,1009,rain,3,0,90,3,-5.681607,144.248908,Papua New Guinea
316,4,4,3,3,3,3,3,4,4,3,...,10,1010,rain,3,0,85,4,-5.681607,144.248908,Papua New Guinea
317,4,4,3,3,3,3,3,4,3,3,...,10,1012,rain,3,0,90,4,-5.681607,144.248908,Papua New Guinea
318,4,4,3,3,3,3,3,4,3,3,...,10,1011,none,3,0,90,3,-5.681607,144.248908,Papua New Guinea


In [14]:
instancia.unir("left", "country", "paises")

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud_x,longitud_x,...,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,latitud_y,longitud_y,paises
0,2018,boating,usa,57.0,white shark,jun,n,f,39.783730,-100.445882,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018,unprovoked,usa,11.0,unespecific,jun,n,f,39.783730,-100.445882,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018,invalid,usa,48.0,unespecific,jun,n,m,39.783730,-100.445882,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018,unprovoked,australia,27.0,unespecific,jun,n,m,-24.776109,134.755000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018,unprovoked,australia,27.0,unespecific,jun,n,m,-24.776109,134.755000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4330,0,unprovoked,south africa,27.0,unespecific,jul,y,m,-28.816624,24.991639,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4331,0,unprovoked,usa,27.0,unespecific,unknown,n,f,39.783730,-100.445882,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4332,0,unprovoked,australia,27.0,tiger shark,unknown,y,m,-24.776109,134.755000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4333,0,unprovoked,australia,27.0,tiger shark,unknown,y,m,-24.776109,134.755000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
